# Mel Schwan, Stuart Miller, Justin Howard, Paul Adams
# Lab Three: Clustering, Association Rules, or Recommenders
## Capstone: Association Rule Mining, Clustering, or Collaborative Filtering

### Lab3 Project Requirments -
1. [Business understanding](#Businessunderstanding)
    1. [Describe the purpose of the data set you selected](#Assessthecurrentsituation)
    2. [Describe how you would define and measure the outcomes from the dataset](#CostBenefit)
    3. [How would you measure the effectiveness of a good prediction algorithm](#Desiredoutputs)
    4. [Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?](#ValMethod)
  
2. [Data Understanding](#Dataunderstanding)
    1. [Describe the meaning and type of data for each attribute in the data file](#Describedata)
    2. [Verify data quality: Explain any missing values, duplicate data, and outliers](#Datareport)
    3. [Give simple, appropriate statistics (range, mode, mean, median, variance, counts, etc.) for the most important attributes and describe what they mean](#Stats)
    4. [Visualize the most important attributes appropriately (at least 5 attributes)](#Visualize)
    5. [Explore relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate](#Correlations)
    6. [Identify and explain interesting relationships between features and the class you are trying to predict](#relationships)
    7. [Are there other features that could be added to the data or created from existing features? Which ones?](#Featurecreation)
    8. [Outlier Removal](#OutlierRemoval)
3. [Modeling and Evaluation](#Model)
    1. [Option A: Cluster Analysis](#Cluster)
    2. [Option B: Association Rule Mining](#Rule_mining)
    2. [Option C: Collaborative Filtering](#Collaborative)
4. [Deployment](#Deployment)
    1. [How useful is your model for interested parties? ](#Useful)
    2. [How would you measure the model's value if it was used by these parties?](#Value)
    3. [How would your deploy your model for interested parties?](#Deploy)
    4. [How often would your model need to be updated?](#Update)
    5. [What other data should be collected? ](#Collect)

A1. [Model Hyperparameter Tuning Details](#A2)


Our project will follow a hybrid methodology, mixing the expectations of the grading rubric with the CRISP_DM framework. CRISP-DM stands for the cross-industry process for data mining, which provides a structured approach to planning a data mining project. It is a robust and well-proven methodology.

In the final assignment for this course, you will be using one of three different analysis methods:
* Option A: Use transaction data for mining associations rules
* Option B: Use clustering on an unlabeled dataset to provide insight or features
* Option C: Use collaborative filtering to build a custom recommendation system
Your choice of dataset will largely determine the task that you are trying to achieve. Though the
dataset does not need to change from your previous tasks. For example, you might choose to use
clustering on your data as a preprocessing step that extracts different features. Then you can use
those features to build a classifier and analyze its performance in terms of accuracy (precision,
recall) and speed. Alternatively, you might choose a completely different dataset and perform rule
mining or build a recommendation system.

<img src="../_images/crisps-dm.png" style="width:550px;height:450px"/>


# 1. Stage One - Business understanding  <a class="anchor" id="Businessunderstanding"></a>


## 1.1 Describe the purpose of the data set you selected (Q1A)<a class="anchor" id="Assessthecurrentsituation"></a>

### 1.1.2. Requirements - <a class="anchor" id="Requirements"></a> 
-  Requirements
  -  Perform a descriptive analysis of a data set: exploring the statistical summaries of the features, visualizing important relationships among the attributes, and making conclusions from the visualizations and analysis. The analytical framework we will follow is a combination of the CRISP-DM framework and the specific requirements laid out within the Machine Learning Lab 1: Visualization.
  -  Demonstrate the team's understanding of ? to best predict ?.
  
- Constraints
  -  Time :  
  -  Data :  




### 1.1.3.Risks and contingencies <a class="anchor" id="Risks"></a>
  -  Outlier assumptions : 
  
  -  Missing data assumptions : 
      -  
      - 
      - 
      
  -  Joined dataset assumptions 

  - Elimintation of features 
       - Elimintation of features can eliminate key factors

### 1.1.4.Terminology <a class="anchor" id="Terminology"></a>
Subject Matter Terminology:
- 


### 1.1.5.Costs and benefits (Q1B) <a class="anchor" id="CostBenefit"></a>
- Cost
    -  

- Benefits
  -  

 ## 1.2 Describe how you would define and measure the outcomes from the dataset (Q1B) <a class="anchor" id="CostBenefit"></a>


**Business success criteria**
- Produce a model capable of predicting 
- Build model capable of predicting whether an 
- Reduce 

**Data mining success criteria**
- Build a data model that has a ROC value of .80 - .90    
- Define successful ?
    - 
    - 


 ## 1.3 Describe how you would define and measure the outcomes from the dataset? (Q1C) <a class="anchor" id="Desiredoutputs"></a>

Our analytic goal is to 

 ## 1.4 Why does your chosen validation method make sense for this specific dataset and the stakeholders needs? (Q1D) <a class="anchor" id="ValMethod"></a>

# 2. Stage  Two - Data Understanding <a class="anchor" id="Dataunderstanding"></a>


## 2.1 Describe the meaning and type of data for each attribute in the data file (Q2A) <a class="anchor" id="Describedata"></a>

Our data comes from the 
Our analysis features the use of several Python libraries, such as Pandas, in addition to a custom data cleaning script foinfo

## 2.2 Verify data quality: Explain any missing values, duplicate data, and outliers (Q2B) <a class="anchor" id="Datareport"></a>



## 2.3 Give simple, appropriate statistics (range, mode, mean, median, variance, counts, etc.) for the most important attributes and describe what they mean (Q2C) <a class="anchor" id="Stats"></a>



### 2.3.1. Missing Data <a class="anchor" id="MissingData"></a>
.

### 2.3.2. Outliers <a class="anchor" id="Outliers"></a>

#### High Values in 

#### Data Cleaning Script

All the cleaning discussed in the sections above are implemented in `cleaning.py`.
This script contains a function (`read_clean_data`) to apply the cleaning steps and return the cleaned dataset for work.

**Details**  
* Cleaning
  * Read csv with Pandas (setting correct data types)
  * Drop columns that will not be used
  * Recode NA values that are not listed as np.nan
  * Formattings
  * Encode categorical variables
* Returns
  * DataFrame with cleaned data


## 2.4 Visualize the most important attributes appropriately (Q2D) <a class="anchor" id="Visualize"></a>

info

-


### 2.4.1 Distributions  <a class="anchor" id="Distributions"></a>

This part of the exploration focused on the use of box plots and histograms for visualing continuous variables and bar charts for visualizing categorical variables. These graphical formats permit the easy identification of skewedness and help us identify outliers. 

Variables we expect to be important were selected for univariate visualization, such as `AMT_INCOME_TOTAL` and `AMT_ANNUITY`. The histogram shows the features of the main distribution, while the behavior of the tails and extreme values are shown in the boxplots. The distribution of incomes is extremely long-tailed and right-skewed, as can be expected with most any income distribution. 

Boxplots indicate a large number of outliers in both the `AMT_TOTAL_INCOME` and `AMT_ANNUITY` feature and one extreme outlier that will require closer exmaination. A comparison of each distribution in its raw and transformed form is displayed below.

## 2.5 Explore relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate (Q2E) <a class="anchor" id="Correlations"></a>


info

## 2.6 Identify and explain interesting relationships between features and the class you are trying to predict (Q2F) <a class="anchor" id="relationships"></a>

infor

## 2.7 Are there other features that could be added to the data or created from existing features (Q2G) <a class="anchor" id="Featurecreation"></a>
Info

## 2.8 Outlier Removal (Q2H)<a class="anchor" id="OutlierRemoval"></a>
#### Feature Selection for Outlier Removal
Outlier removal was performed following feature analysis, imputation, and engineering to allow for a complete assessment of the raw data. The outlier
removal process required to first identify the order of feature importance. To perform this, we applied a Random Forest and used the resutling Gini 
coefficients to identify importance. Categorical features were one-hot encoded prior to constructing a random forest. While there may be some potential minimization of categorical importance from this approach (through introducing more parameters and slightly dilluting the correlation), any residual collinearity following variable elimination would be likewise minimized by further separating significance of categorical variables in preference to other collinear, continuous and nominally scaled integer features.

Using the random forest output, the top 15 features were identified. We decided to analyze 15 features for outlier removal due to the exponentially decaying significance of subsequent features to the target and the negative impact of removing too many observations. We performed analysis on these 15 features using histograms to assess dispersion and measured skewness on all transformed features to identify the most practical transformations (root, logarithmic, squared) required for normalization. Skewness around +/-1 was considered extreme, between +/-0.5 as moderate, and 0 as non-existent. Of these 15 features analyzed, eight features had outliers removed; highly imputed features and those with 

#### Three Standard Deviations
Once normalized, features were then stripped of outliers. The outlier treatment applied uses z-tests to assess statistical dispersion of the data. Values beyond +/- three standard deviations are dropped from the model. This approach was selected because three standard deviations covers roughly 99.78% of all data, eliminating only extreme outliers. Three standard deviations is also within a reliable range for hypothesis testing. After dropping outlying observations from the data, we then back-transformed it to allow prior knowledge in the data to remain for its use in classification modeling.

#### Quantifying Outlier Removal Impact
Prior to outlier removal, the dataset had 307,511 observations. Following removal, the volume decreased by 21.399 percent to a count of 241,744 observations.

# 3. Stage  Three - Modeling and Evaluation <a class="anchor" id="Model"></a>

## 3.1 Option A: Cluster Analysis (Q3A)<a class="anchor" id="Cluster"></a>

Info

## 3.2 Option B: Association Rule Mining (Q3B)<a class="anchor" id="Rule_mining"></a>

Info

## 3.3 Option C: Collaborative Filtering (Q3C)<a class="anchor" id="Collaborative"></a>

Info

# 4. Stage Five - Deployment (Q3) <a class="anchor" id="Deployment"></a>

## 4.1 Next Stage Deployment <a class="anchor" id="Deployment"></a>

The next stage in the CRISP-DM is deployment. After model building and evaluation, we are ready to deploy our code representation of the model into a production environment and solve our original business problem.
Our business problem is to give Home Credit loan evaluators access to a model that evaluates an applicant’s current and past financial history in determining whether to approve the requested loan.

#### How useful is your model for interested parties?<a class="anchor" id="Useful"></a>

We believe this model would be useful for loan departments loan evaluators. This is contingent of discovering how some of the external variables are created. With this and some additional consistancy in top scoring features we could improve the accuracy

#### How would you measure the model's value if it was used by these parties?<a class="anchor" id="Value"></a>

This model should be tested in parrellel to present evaluation proceess. Then after a set period of time compare human evaluation to model based accuracy. If the results were the same, the minimum resulting savings would be the salaries of the loan evaluators. Added value would result from the acceleration of the loan approval process.

#### How would your deploy your model for interested parties?<a class="anchor" id="Deploy"></a>

Depending on the resources,  available, models can be deployed as batch or real-time predictions. Home Credits current process is a batch implementation. The applicant fills out the form which is then digitized and sent to the loan approval department. During the loan approval, the collected data will need to be cleaned and normalized before processed through the machine learning predictive model.

#### How often would the model need to be updated?<a class="anchor" id="Update"></a>

The metrics for the customer is continually updated. On a scheduled cycle, this new dataset is analyzed through the current model. Doing batch cycles will allow for consistency in whether an applicant is approved or not.  However, model designs will change as newer technology are available or changing business environments make reengineering required.

#### What other data should be collected?<a class="anchor" id="Collect"></a>

In addition to the clarification of data we have described above, there would be a tremendous value in the financials for the industry that the individual is working. These financials could assist in prediction of any future concerns for the applicants present income.

Below is a typical example of a batch deployment \[1\].

<img src="../_images/batchdeployment_process.png" style="width:800px;height:375px"/>



## References

\[1\] J. Kervizic, Overview of Different Approaches to Deploying Machine Learning Models in Production, June 2016.
Accessed on: Feb. 15, 2019. \[Online\].
Available: https://www.kdnuggets.com/2019/06/approaches-deploying-machine-learning-production.html

# A1. Parameter Tuning <a class="anchor" id="A2"></a>

The code and output of model tuneing in shown below.